<a href="https://colab.research.google.com/github/vbonato/cnnTestBench/blob/main/cnnTestBench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# Updated CNN Code Generator (SRC/BIN Structure) - V3.0: Fixed Global Dimension

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import os
import math

# --- 1. Core Code Generation Functions (Updated for Global DIM) ---

def generate_makefile_code(num_layers):
    """Generates the Makefile content. (No change)"""
    TARGET = f'conv{num_layers}'
    TB_OBJ = f'conv_tb{num_layers}.o'
    TB_SRC = f'conv_tb{num_layers}.cpp'
    TB_HDR = f'conv_tb{num_layers}.h'
    CNN_HDR = f'conv{num_layers}.h'
    CNN_SRC = f'conv{num_layers}.cpp'

    makefile_lines = [
        f'{TARGET}: check_dirs {TARGET}.o {TB_OBJ}',
        f'\tclang++ {TARGET}.o {TB_OBJ} -o ../bin/{TARGET} -lm\n',

        f'{TB_OBJ}: {TB_SRC} {TB_HDR} {CNN_HDR}',
        f'\tclang++ -c {TB_SRC} -o {TB_OBJ}\n',

        f'{TARGET}.o: {CNN_SRC} {CNN_HDR}',
        f'\tclang++ -c {CNN_SRC} -o {TARGET}.o\n',

        'check_dirs:',
        '\t@mkdir -p ../bin\n',

        '.PHONY: clean',
        'clean:',
        f'\trm -f {TARGET}.o {TB_OBJ}',
        f'\trm -f ../bin/{TARGET}',
        f'\trm -f ../bin/output.bin\n'
    ]
    return '\n'.join(makefile_lines)


def generate_testbench_header_code(num_layers):
    """Generates the testbench header C++ code (conv_tbX.h). (No change)"""
    code = f"#ifndef CONV_TB_H\n#define CONV_TB_H\n\n"
    code += f"// ** GLOBAL TESTBENCH CONSTANTS **\n"
    code += f"const int RANDROOF = 256; // Max value for randomized input/weights/bias\n"
    code += f"\n#endif"
    return code

def generate_testbench_code(num_layers, params):
    """Generates the testbench C++ code, now using GLOBAL_DIM."""
    N_CLASSES = params['N_CLASSES']
    GLOBAL_DIM = params['GLOBAL_DIM']

    code_lines = [
        '#include <cstdio>',
        '#include <cstdlib>',
        '#include <iostream>',
        '#include <cmath>',
        f'#include "conv_tb{num_layers}.h"',
        f'#include "conv{num_layers}.h"\n',
        'int main(void) {'
    ]

    # --- Memory Allocation ---

    # Input Buffer (uses GLOBAL_DIM)
    H1_size = f"H1 * H1" if GLOBAL_DIM == 2 else f"H1"

    code_lines.append(f' \t// Input Buffer (C1 x H1 x (H1 if 2D) | {GLOBAL_DIM}D)')
    code_lines.append(f' \ttype_t *I1 = (type_t *) malloc(C1 * {H1_size} * sizeof(type_t));')

    # Weight Buffers (uses GLOBAL_DIM)
    code_lines.append(f' \t// Weight Buffers for {num_layers} Conv Layers')
    for i in range(1, num_layers + 1):
        # R_i is squared for 2D, linear for 1D
        R_size = f" * R{i}" if GLOBAL_DIM == 2 else ""
        code_lines.append(f' \ttype_t *W{i} = (type_t *) malloc(M{i} * C{i} * R{i}{R_size} * sizeof(type_t));')

    # Bias Buffers (unchanged)
    code_lines.append(f' \t// Bias Buffers for {num_layers} Conv Layers')
    for i in range(1, num_layers + 1):
        code_lines.append(f' \ttype_t *B{i} = (type_t *) malloc(M{i} * sizeof(type_t));')

    # FC Weight Buffer (Input size depends on GLOBAL_DIM)
    code_size_n = f"M{num_layers} * E{num_layers}"
    if GLOBAL_DIM == 2:
        code_size_n += f" * E{num_layers}"

    code_lines.append(f' \t// Weight Buffer for Final FC Layer (Input size: {code_size_n})')
    code_lines.append(f' \ttype_t *W_fc = (type_t *) malloc({code_size_n} * N_CLASSES * sizeof(type_t));')

    # FC Bias Buffer & Final Output Buffer (unchanged)
    code_lines.append(f' \t// Bias Buffer for Final FC Layer')
    code_lines.append(f' \ttype_t *B_fc = (type_t *) malloc(N_CLASSES * sizeof(type_t));')
    code_lines.append(f' \t// Final Output Buffer (Softmax probabilities)')
    code_lines.append(f' \tfloat *O_final = (float *) calloc(N_CLASSES, sizeof(float));')

    # --- Initialization ---
    code_lines.append('\n \tsrand(1);')
    code_lines.append(f' \t// Initialize Input I1')
    code_lines.append(f' \tfor(unsigned j = 0; j < C1 * {H1_size}; j++)')
    code_lines.append(' \t\tI1[j] = rand() % RANDROOF;')

    # Init Conv Weights and Biases (uses GLOBAL_DIM)
    code_lines.append(f'\n \t// Initialize Conv Weights and Biases')
    for i in range(1, num_layers + 1):
        R_size = f" * R{i}" if GLOBAL_DIM == 2 else ""
        code_lines.append(f' \tfor(unsigned j = 0; j < M{i} * C{i} * R{i}{R_size}; j++)')
        code_lines.append(f' \t\tW{i}[j] = rand() % RANDROOF;')
        code_lines.append(f' \tfor(unsigned j = 0; j < M{i}; j++)')
        code_lines.append(f' \t\tB{i}[j] = rand() % RANDROOF;')

    # Init FC Weights and Biases (unchanged)
    code_lines.append(f'\n \t// Initialize FC Weights and Biases')
    code_lines.append(f' \tfor(unsigned j = 0; j < {code_size_n} * N_CLASSES; j++)')
    code_lines.append(f' \t\tW_fc[j] = rand() % RANDROOF;')
    code_lines.append(f' \tfor(unsigned j = 0; j < N_CLASSES; j++)')
    code_lines.append(f' \t\tB_fc[j] = rand() % RANDROOF;')

    # CNN Call, Prediction, and Freeing (Logic remains the same)
    code_lines.append('\n \t// Perform CNN inference')
    cnn_call = ' \tcnn(I1'
    for i in range(1, num_layers + 1):
        cnn_call += f', W{i}, B{i}'
    cnn_call += f', W_fc, B_fc, O_final);'
    code_lines.append(cnn_call)

    # --- Prediction and Print Logic (unchanged) ---
    code_lines.append('\n \t// Find the maximum probability (the prediction)')
    code_lines.append(' \tfloat max_val = O_final[0];')
    code_lines.append(' \tint predicted_class = 0;')
    code_lines.append(' \tfor (int k = 1; k < N_CLASSES; k++) {')
    code_lines.append(' \t\tif (O_final[k] > max_val) {')
    code_lines.append(' \t\t\tmax_val = O_final[k];')
    code_lines.append(' \t\t\tpredicted_class = k;')
    code_lines.append(' \t\t}')
    code_lines.append(' \t}')
    code_lines.append('\n \tprintf("--- Classification Result (Softmax Probabilities) ---\\n");')
    code_lines.append(' \tprintf("Predicted Class Index: %d (with probability %.4f)\\n", predicted_class, max_val);')
    code_lines.append(' \tprintf("All Probabilities:\\n");')
    code_lines.append(' \tfor (int k = 0; k < N_CLASSES; k++) {')
    code_lines.append(' \t\tprintf(" \tClass %d: %.4f\\n", k, O_final[k]);')
    code_lines.append(' \t}')
    code_lines.append(' \tprintf("---------------------------------------------------\\n");')

    # Output to file
    code_lines.append(f'\n \tFILE *opf = fopen("../bin/output.bin", "wb");')
    code_lines.append(f' \tif (opf == NULL) {{')
    code_lines.append(f' \t\tperror("Error opening output.bin");')
    code_lines.append(f' \t\treturn EXIT_FAILURE;')
    code_lines.append(f' \t}}')
    code_lines.append(f' \tfwrite(O_final, sizeof(float), N_CLASSES, opf);')
    code_lines.append(' \tfclose(opf);')

    # Free memory
    code_lines.append('\n \t// Free allocated memory')
    code_lines.append(' \tif(I1) free(I1);')
    for i in range(1, num_layers + 1):
        code_lines.append(f' \tif(W{i}) free(W{i});')
        code_lines.append(f' \tif(B{i}) free(B{i});')

    code_lines.append(' \tif(W_fc) free(W_fc);')
    code_lines.append(' \tif(B_fc) free(B_fc);')
    code_lines.append(' \tif(O_final) free(O_final);')

    code_lines.append('\n \treturn EXIT_SUCCESS;\n}')
    return '\n'.join(code_lines)


def generate_conv_code(num_layers, params):
    """Generates the convolution and wrapper C++ implementation code, with specialized 1D and 2D functions."""
    GLOBAL_DIM = params['GLOBAL_DIM']

    code = f'#include "conv{num_layers}.h"\n#include <math.h>\n\n'
    code += "#include <stdio.h>\n\n"

    # ReLU function
    code += "type_t relu(type_t x) {\n"
    code += " \treturn (x > 0) ? x : 0;\n"
    code += "}\n\n"

    # Softmax function (unchanged)
    code += "void softmax(type_t input[N_CLASSES], float output[N_CLASSES]) {\n"
    code += " \tfloat sum = 0.0f;\n"
    code += " \tfloat max_val = (float)input[0];\n"
    code += " \tfor (int k = 1; k < N_CLASSES; k++) {\n"
    code += " \t\tif ((float)input[k] > max_val) max_val = (float)input[k];\n"
    code += " \t}\n"
    code += " \tfor (int k = 0; k < N_CLASSES; k++) {\n"
    code += " \t\toutput[k] = expf((float)input[k] - max_val);\n"
    code += " \t\tsum += output[k];\n"
    code += " \t}\n"
    code += " \tfor (int k = 0; k < N_CLASSES; k++) {\n"
    code += " \t\toutput[k] /= sum;\n"
    code += " \t}\n"
    code += "}\n\n"

    # --- GENERATE SPECIALIZED 1D CONV FUNCTIONS ---
    for i in range(1, num_layers + 1):
        # I[C*H], W[M*C*R], B[M], O[M*E]
        code += f"// ** Specialized 1D Convolution Layer {i} **\n"
        code += f"void conv_1d_{i}(type_t I{i}[C{i} * H{i}], type_t W{i}[M{i} * C{i} * R{i}], type_t B{i}[M{i}], type_t O{i}[M{i} * E{i}]) {{\n"
        code += f" \tfor(int m = 0; m < M{i}; m++) {{\n"
        code += f" \t\tfor(int x = 0; x < E{i}; x++) {{\n"

        code += f" \t\t\tO{i}[x + m * E{i}] = B{i}[m];\n"
        code += f" \t\t\t#pragma HLS PIPELINE II=1\n"

        code += f" \t\t\tfor(int c = 0; c < C{i}; c++) {{\n"
        code += f" \t\t\t\tfor(int l = 0; l < R{i}; l++) {{\n"

        code += f" \t\t\t\t\tint h2 = x * S{i} - PAD{i} + l;\n"
        code += f" \t\t\t\t\ttype_t val = (h2 < 0 || h2 >= H{i}) ? 0 : I{i}[h2 + c * H{i}];\n"
        code += f" \t\t\t\t\tO{i}[x + m * E{i}] += val * W{i}[l + c * R{i} + m * C{i} * R{i}];\n"

        code += f" \t\t\t\t}}\n"
        code += f" \t\t\t}}\n"
        code += f" \t\t}}\n"
        code += f" \t}}\n"
        code += f"}}\n\n"


    # --- GENERATE SPECIALIZED 2D CONV FUNCTIONS ---
    for i in range(1, num_layers + 1):
        # I[C*H*H], W[M*C*R*R], B[M], O[M*E*E]
        code += f"// ** Specialized 2D Convolution Layer {i} **\n"
        code += f"void conv_2d_{i}(type_t I{i}[C{i} * H{i} * H{i}], type_t W{i}[M{i} * C{i} * R{i} * R{i}], type_t B{i}[M{i}], type_t O{i}[M{i} * E{i} * E{i}]) {{\n"
        code += f" \tfor(int m = 0; m < M{i}; m++) {{\n"
        code += f" \t\tfor(int y = 0; y < E{i}; y++) {{\n"
        code += f" \t\t\tfor(int x = 0; x < E{i}; x++) {{\n"

        code += f" \t\t\t\tO{i}[x + (y + (m * E{i})) * E{i}] = B{i}[m];\n"
        code += f" \t\t\t\t#pragma HLS PIPELINE II=1\n"

        code += f" \t\t\t\tfor(int c = 0; c < C{i}; c++) {{\n"
        code += f" \t\t\t\t\tfor(int k = 0; k < R{i}; k++) {{\n"
        code += f" \t\t\t\t\t\tfor(int l = 0; l < R{i}; l++) {{\n"

        code += f" \t\t\t\t\t\t\tint h1 = y * S{i} - PAD{i} + k;\n"
        code += f" \t\t\t\t\t\t\tint h2 = x * S{i} - PAD{i} + l;\n"
        code += f" \t\t\t\t\t\t\ttype_t val = (h1 < 0 || h1 >= H{i} || h2 < 0 || h2 >= H{i}) ? 0 : I{i}[h2 + (h1 + (c * H{i})) * H{i}];\n"
        code += f" \t\t\t\t\t\t\tO{i}[x + (y + (m * E{i})) * E{i}] += val * W{i}[l + (k + (c + (m * C{i})) * R{i}) * R{i}];\n"

        code += f" \t\t\t\t\t\t}}\n"
        code += f" \t\t\t\t\t}}\n"
        code += f" \t\t\t\t}}\n"
        code += f" \t\t\t}}\n"
        code += f" \t\t}}\n"
        code += f" \t}}\n"
        code += f"}}\n\n"


    # Final Dedicated FC layer (input size depends on GLOBAL_DIM)
    input_size_n = f"M{num_layers} * E{num_layers}"
    if GLOBAL_DIM == 2:
        input_size_n += f" * E{num_layers}"

    code += f"// ** Fully Connected Layer **\n"
    code += f"void fc_layer(type_t input[{input_size_n}], type_t W_fc[{input_size_n} * N_CLASSES], type_t B_fc[N_CLASSES], type_t output[N_CLASSES]) {{\n"
    code += f" \tfor (int k = 0; k < N_CLASSES; k++) {{\n"
    code += f" \t\toutput[k] = B_fc[k];\n"
    code += f" \t\tfor (int i = 0; i < {input_size_n}; i++) {{\n"
    code += f" \t\t\toutput[k] += input[i] * W_fc[i + k * {input_size_n}];\n"
    code += f" \t\t}}\n"
    code += f" \t}}\n"
    code += f"}}\n\n"

    # --- WRAPPER CNN FUNCTION (Using Compile-Time Selection via GLOBAL_DIM) ---
    code += "// ** Wrapper CNN Function **\n"
    code += "void cnn(type_t *input"
    for i in range(1, num_layers + 1):
        code += f", type_t *W{i}, type_t *B{i}"
    code += ", type_t *W_fc, type_t *B_fc, float *output) {\n"
    code += "#pragma HLS DATAFLOW\n"

    # Intermediate buffers (sizes adjusted based on GLOBAL_DIM)
    for i in range(1, num_layers + 1):
        E_size = f" * E{i}" if GLOBAL_DIM == 2 else ""
        code += f" \tstatic type_t O{i}[M{i} * E{i}{E_size}];\n"

    code += f" \tstatic type_t O_raw[N_CLASSES];\n\n"

    # Chaining layers using C preprocessor conditionals based on GLOBAL_DIM
    for i in range(1, num_layers + 1):
        # Determine the input and output buffer size for ReLU loop
        O_size = f"M{i} * E{i}"
        if GLOBAL_DIM == 2:
            O_size += f" * E{i}"

        I_name = "input" if i == 1 else f"O{i-1}"

        code += f"\n \t// --- Layer {i} ({GLOBAL_DIM}D) ---\n"
        code += f" #if GLOBAL_DIM == 1\n"
        code += f" \tconv_1d_{i}({I_name}, W{i}, B{i}, O{i});\n"
        code += f" #elif GLOBAL_DIM == 2\n"
        code += f" \tconv_2d_{i}({I_name}, W{i}, B{i}, O{i});\n"
        code += f" #endif\n"

        # Apply ReLU
        code += f" \t// Apply ReLU\n"
        code += f" \tfor (int j = 0; j < {O_size}; j++) O{i}[j] = relu(O{i}[j]);\n"


    # Final FC and Softmax layers
    code += "\n \t// Final Layer: Fully Connected (FC) Classification Head\n"
    code += f" \tfc_layer(O{num_layers}, W_fc, B_fc, O_raw);\n"

    code += "\n \t// Softmax Layer: Convert raw scores to probability distribution\n"
    code += f" \tsoftmax(O_raw, output);\n"

    code += "}\n"
    return code


def generate_convh_code(num_layers, params):
    """Generates the convolution header C++ code with calculated parameters, including GLOBAL_DIM constant."""
    GLOBAL_DIM = params['GLOBAL_DIM']

    code = "#ifndef CONV_H\n#define CONV_H\n\n#include <cstddef>\n#include <stdio.h>\n\ntypedef unsigned type_t;\n\n"

    # Global Parameters
    code += f"// ** GLOBAL PARAMETERS **\n"
    code += f"const size_t N_CLASSES = {params['N_CLASSES']};\n"
    code += f"const size_t GLOBAL_DIM = {GLOBAL_DIM}; // All layers are {GLOBAL_DIM}D\n"
    code += f"const size_t BIAS_ON = 1;\n\n"

    # Input Parameters (H1 is the 1D dimension, or one side of a square 2D image)
    code += "// ** INPUT PARAMETERS **\n"
    code += f"const size_t C1 = {params['C1']}; // Input channels\n"
    code += f"const size_t H1 = {params['H1']}; // Input size (seq length or side length H) \n\n"

    for i in range(1, num_layers + 1):
        # Determine the C and H variable names
        if i == 1:
            C_name = 'C1'
            H_name = 'H1'
        else:
            # For all other layers, C and H are their own indexed variables
            C_name = f'C{i}'
            H_name = f'H{i}'

        code += f"// ** CONV LAYER {i} **\n"
        # Layer-specific constants (only M, R, S, E, F, PAD are always indexed)
        # We reuse the C1/H1 definitions for layer 1, and use C2/H2, etc. for the rest.
        if i > 1:
             code += f"const size_t {C_name} = {params[C_name]};\n" # Input Channels
             code += f"const size_t {H_name} = {params[H_name]};\n" # Input Size

        code += f"const size_t M{i} = {params[f'M{i}']};\n" # Output Channels
        code += f"const size_t R{i} = {params[f'R{i}']};\n" # Kernel Size
        code += f"const size_t S{i} = {params[f'S{i}']};\n" # Stride
        code += f"const size_t E{i} = {params[f'E{i}']};\n" # Output Size

        # Calculated parameters (Padded size and Pad amount)
        code += f"const size_t F{i} = {params[f'F{i}']};\n"
        code += f"const size_t PAD{i} = {params[f'PAD{i}']};\n\n"

    # ... [Rest of the function remains the same: Function prototypes, etc.]
    # Function prototypes (Specialized 1D and 2D functions are always generated for flexibility)
    for i in range(1, num_layers + 1):
        # 1D Conv Prototype
        code += f"void conv_1d_{i}(type_t I{i}[C{i} * H{i}], type_t W{i}[M{i} * C{i} * R{i}], type_t B{i}[M{i}], type_t O{i}[M{i} * E{i}]);\n"
        # 2D Conv Prototype
        code += f"void conv_2d_{i}(type_t I{i}[C{i} * H{i} * H{i}], type_t W{i}[M{i} * C{i} * R{i} * R{i}], type_t B{i}[M{i}], type_t O{i}[M{i} * E{i} * E{i}]);\n"

    # FC and Softmax prototypes
    input_size_n = f"M{num_layers} * E{num_layers}"
    if GLOBAL_DIM == 2:
        input_size_n += f" * E{num_layers}"

    code += f"void fc_layer(type_t input[{input_size_n}], type_t W_fc[{input_size_n} * N_CLASSES], type_t B_fc[N_CLASSES], type_t output[N_CLASSES]);\n"
    code += f"void softmax(type_t input[N_CLASSES], float output[N_CLASSES]);\n"

    # CNN prototype (final output is float*)
    code += "\nvoid cnn(type_t *input"
    for i in range(1, num_layers + 1):
        code += f", type_t *W{i}, type_t *B{i}"
    code += ", type_t *W_fc, type_t *B_fc, float *output);\n"

    code += "\n#endif"
    return code

# --- 2. Parameter Calculation and GUI Setup (Simplified) ---

def calculate_output_size(H_in, R, S, P):
    """Calculates output size E based on HLS-style padding/stride. We enforce E = ceil(H_in / S)"""
    return math.ceil(H_in / S)

def generate_parameter_widgets(num_layers):
    """Generates the parameter input widgets with a single global dimension selector."""

    # Global Parameters
    n_classes_widget = widgets.IntText(value=10, description='N_CLASSES:', min=1, style={'description_width': 'initial'})

    # Global Dimension Selector
    global_dim_widget = widgets.Dropdown(options=[(1, 1), (2, 2)], value=2, description='GLOBAL DIMENSION:', style={'description_width': 'initial'})
    bias_on_widget = widgets.IntText(value=1, description='Bias ON (1=Yes):', min=1, max=1, style={'description_width': 'initial'}, disabled=True)

    # Input Feature Map (Layer 1 Input)
    c1_widget = widgets.IntText(value=3, description='Input C1 (Channels):', min=1, style={'description_width': 'initial'})
    h1_widget = widgets.IntText(value=32, description='Input H1 (Size):', min=1, style={'description_width': 'initial'})

    widgets_list = [
        widgets.HTML(value="<h3>Global Configuration:</h3>"),
        global_dim_widget,
        widgets.HBox([n_classes_widget, bias_on_widget]),
        widgets.HTML(value="<h3>Input Feature Map (Layer 1 Input):</h3>"),
        c1_widget,
        h1_widget,
        widgets.HTML(value=f"<h3>Convolution Layers (1 to {num_layers}):</h3>")
    ]

    layer_widgets = {}

    for i in range(1, num_layers + 1):
        # User defined parameters (Dimension is now fixed globally)
        m_i = widgets.IntText(value=32, description=f'L{i} M (Ch Out):', min=1, style={'description_width': 'initial'})
        r_i = widgets.IntText(value=3, description=f'L{i} R (Kernel):', min=1, style={'description_width': 'initial'})
        s_i = widgets.IntText(value=1, description=f'L{i} S (Stride):', min=1, style={'description_width': 'initial'})

        layer_widgets[i] = (m_i, r_i, s_i)

        widgets_list.append(widgets.VBox([
            widgets.HTML(value=f"<h4>Conv Layer {i}</h4>"),
            widgets.HBox([m_i, r_i, s_i])
        ]))

    params_vbox = widgets.VBox(widgets_list)

    return params_vbox, layer_widgets, c1_widget, h1_widget, n_classes_widget, global_dim_widget

def collect_and_calculate_params(num_layers, layer_widgets, c1_widget, h1_widget, n_classes_widget, global_dim_widget):
    """Collects user input and calculates dependent parameters, using a single GLOBAL_DIM."""

    params = {}

    # Global Parameters
    params['N_CLASSES'] = n_classes_widget.value
    params['GLOBAL_DIM'] = global_dim_widget.value
    GLOBAL_DIM = params['GLOBAL_DIM']

    # Initial Input (Layer 1 Input)
    params['C1'] = c1_widget.value
    params['H1'] = h1_widget.value

    H_prev = params['H1']
    M_prev = params['C1']

    for i in range(1, num_layers + 1):
        # M_i, R_i, S_i are retrieved from the widgets
        m_i, r_i, s_i = layer_widgets[i]

        params[f'M{i}'] = m_i.value
        params[f'R{i}'] = r_i.value
        params[f'S{i}'] = s_i.value

        # --- Calculate Input C_i and H_i for Layer i ---
        params[f'C{i}'] = M_prev # Channels is always the previous layer's M

        if i == 1:
            params[f'H{i}'] = params['H1'] # H1 is defined at start
        else:
            # H_i depends on E_{i-1}
            if GLOBAL_DIM == 2:
                # 2D -> 2D: H_i is E_{i-1}
                params[f'H{i}'] = H_prev
            else:
                # 1D -> 1D: H_i is E_{i-1}
                params[f'H{i}'] = H_prev

        H_curr = params[f'H{i}']

        # Calculated Output Size (E_i)
        params[f'E{i}'] = calculate_output_size(H_curr, params[f'R{i}'], params[f'S{i}'], 0)

        # Calculated Padding and Padded Size
        E_curr = params[f'E{i}']
        F_i = (E_curr * params[f'S{i}'] + params[f'R{i}'] - 1)
        params[f'F{i}'] = F_i
        params[f'PAD{i}'] = (F_i - H_curr) // 2

        # --- Setup for next layer (i+1) ---
        H_prev = E_curr
        M_prev = params[f'M{i}']

        # Validation (Checks based on GLOBAL_DIM)
        if params[f'R{i}'] > H_curr and H_curr > 1:
            raise ValueError(f"Layer {i}: Kernel size R{i} must be less than or equal to input size H{i}.")
        if params['N_CLASSES'] < 1:
            raise ValueError("N_CLASSES must be 1 or greater.")

        # 2D Specific Validation
        if GLOBAL_DIM == 2 and params[f'R{i}'] % 2 == 0:
             raise ValueError(f"Layer {i}: 2D kernel R{i} must be odd to ensure symmetric padding in HLS style (R=3, 5, 7...).")

    return params


# --- 3. GUI and Execution ---

layer_slider = widgets.IntSlider(value=3, min=1, max=5, step=1, description='CNN Layers:', continuous_update=False, style={'description_width': 'initial'})
generate_button = widgets.Button(description='Generate Code (Fixed DIM HLS + Softmax)', button_style='success')
output_area = widgets.Output()
code_controls_vbox = widgets.VBox()

# Initial setup of parameter widgets
param_vbox, layer_widgets_map, c1_input, h1_input, n_classes_input, global_dim_input = generate_parameter_widgets(layer_slider.value)
code_controls_vbox.children = (param_vbox,)

def update_widgets(change):
    """Update parameter widgets when the number of layers changes."""
    global layer_widgets_map, c1_input, h1_input, n_classes_input, global_dim_input

    num_layers = layer_slider.value
    new_param_vbox, layer_widgets_map, c1_input, h1_input, n_classes_input, global_dim_input = generate_parameter_widgets(num_layers)

    code_controls_vbox.children = (new_param_vbox,)

    with output_area:
        clear_output()
        display(Markdown(f"Parameters updated for **{num_layers} layers**. Click 'Generate Code'."))

layer_slider.observe(update_widgets, names='value')

def on_button_click(b):
    """Code generation main logic."""
    with output_area:
        clear_output()

        num_layers = layer_slider.value
        i = num_layers

        try:
            # 1. Collect and Calculate Parameters
            params = collect_and_calculate_params(num_layers, layer_widgets_map, c1_input, h1_input, n_classes_input, global_dim_input)
            GLOBAL_DIM = params['GLOBAL_DIM']

            # 2. Setup Folder
            root_folder = f'./repo/generatedCNN_{i}Layers_{GLOBAL_DIM}D'
            source_folder = os.path.join(root_folder, 'src')
            bin_folder = os.path.join(root_folder, 'bin')
            os.makedirs(source_folder, exist_ok=True)
            os.makedirs(bin_folder, exist_ok=True)

            # 3. Generate Files
            with open(f'{source_folder}/Makefile{i}', 'w') as f: f.write(generate_makefile_code(i))
            with open(f'{source_folder}/conv_tb{i}.cpp', 'w') as f: f.write(generate_testbench_code(i, params))
            with open(f'{source_folder}/conv_tb{i}.h', 'w') as f: f.write(generate_testbench_header_code(i))
            with open(f'{source_folder}/conv{i}.h', 'w') as f: f.write(generate_convh_code(i, params))
            with open(f'{source_folder}/conv{i}.cpp', 'w') as f: f.write(generate_conv_code(i, params))

            display(Markdown(f'## ✅ Success! All files for **{i} {GLOBAL_DIM}D CONV layers** + **1 FC layer (with Softmax)** generated in `{source_folder}`'))

            # Print a summary of the calculated parameters
            param_summary = [f"| Layer | Type | D (Dim) | C (In) | H (In) | M (Out) | R (K) | S (Str) | E (Out) | Activation |"]
            param_summary.append("|---|---|---|---|---|---|---|---|---|---|")

            # Initial Input
            H1_display = f"{params['H1']}x{params['H1']}" if GLOBAL_DIM == 2 else f"{params['H1']}"
            param_summary.append(f"| Input | Image | **{GLOBAL_DIM}** | {params['C1']} | {H1_display} | N/A | N/A | N/A | N/A | No |")

            for j in range(1, num_layers + 1):
                H_display = f"{params[f'H{j}']}x{params[f'H{j}']}" if GLOBAL_DIM == 2 else f"{params[f'H{j}']}"
                E_display = f"{params[f'E{j}']}x{params[f'E{j}']}" if GLOBAL_DIM == 2 else f"{params[f'E{j}']}"

                param_summary.append(f"| {j} | CONV | **{GLOBAL_DIM}** | {params[f'C{j}']} | {H_display} | {params[f'M{j}']} | {params[f'R{j}']} | {params[f'S{j}']} | {E_display} | **ReLU** |")

            # Final FC layer summary
            input_size = params[f'M{num_layers}'] * params[f'E{num_layers}']
            if GLOBAL_DIM == 2:
                input_size *= params[f'E{num_layers}']

            param_summary.append(f"| N+1 | **FC** | N/A | {input_size} (flat) | N/A | **{params['N_CLASSES']}** | N/A | N/A | N/A | **Softmax** |")

            display(Markdown('### Calculated CNN Parameters\n' + '\n'.join(param_summary)))

            display(Markdown('### Execution Instructions (SRC/BIN Structure)'))
            display(Markdown(
                f"1. **Navigate to the Source:** `cd {source_folder}`\n"
                f"2. **Compile:** `make -f Makefile{i} conv{i}` (The specialized {GLOBAL_DIM}D code path is selected at compile time.)\n"
                f"3. **Run:** Go to the parent directory: `cd ../` and execute the binary: `./bin/conv{i}`\n"
            ))

        except ValueError as e:
            display(Markdown(f'## ❌ Error: {e}'))

        except Exception as e:
            display(Markdown(f'## ❌ An unexpected error occurred: {e}'))


generate_button.on_click(on_button_click)

# FINAL DISPLAY: Display the components only once
display(layer_slider, code_controls_vbox, generate_button, output_area)

IntSlider(value=3, continuous_update=False, description='CNN Layers:', max=5, min=1, style=SliderStyle(descrip…

Button(button_style='success', description='Generate Code (Fixed DIM HLS + Softmax)', style=ButtonStyle())

Output()

In [ ]:
xxxxx

# Updated CNN Code Generator (SRC/BIN Structure)

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import os
import math

# --- 1. Core Code Generation Functions (Updated for Softmax) ---

def generate_makefile_code(num_layers):
    """
    Generates the Makefile content, updated to use clang++ for all C++ files.
    """
    TARGET = f'conv{num_layers}'
    TB_OBJ = f'conv_tb{num_layers}.o'
    TB_SRC = f'conv_tb{num_layers}.cpp'
    TB_HDR = f'conv_tb{num_layers}.h'
    CNN_HDR = f'conv{num_layers}.h'
    CNN_SRC = f'conv{num_layers}.cpp'

    makefile_lines = [
        # The ultimate target: creates dir, links objects, and moves executable
        f'{TARGET}: check_dirs {TARGET}.o {TB_OBJ}',
        # Compile and move the executable one directory up into the bin folder
        # clang++ is used for C++ linking
        f'\tclang++ {TARGET}.o {TB_OBJ} -o ../bin/{TARGET} -lm\n', # Added -lm for math library

        # Rule for the Testbench Object File
        f'{TB_OBJ}: {TB_SRC} {TB_HDR} {CNN_HDR}',
        # clang++ is used for C++ compilation
        f'\tclang++ -c {TB_SRC} -o {TB_OBJ}\n',

        # Rule for the CNN Object File
        f'{TARGET}.o: {CNN_SRC} {CNN_HDR}',
        # clang++ is used for C++ compilation
        f'\tclang++ -c {CNN_SRC} -o {TARGET}.o\n',

        # NEW TARGET: Checks and creates the sibling bin/ folder (one level up)
        'check_dirs:',
        '\t@mkdir -p ../bin\n',

        '.PHONY: clean',
        'clean:',
        # Delete object files from the current folder (src/)
        f'\trm -f {TARGET}.o {TB_OBJ}',
        # Delete the final executable and output file from the sibling bin/ folder
        f'\trm -f ../bin/{TARGET}',
        f'\trm -f ../bin/output.bin\n'
    ]
    return '\n'.join(makefile_lines)


def generate_testbench_header_code(num_layers):
    """Generates the testbench header C++ code (conv_tbX.h). (No change needed)"""
    # Defines constants needed only by the testbench, like RANDROOF.
    code = f"#ifndef CONV_TB_H\n#define CONV_TB_H\n\n"
    code += f"// ** GLOBAL TESTBENCH CONSTANTS **\n"
    code += f"const int RANDROOF = 256; // Max value for randomized input/weights/bias\n"
    code += f"\n#endif"
    return code

def generate_testbench_code(num_layers, params):
    """
    Generates the testbench C++ code, updated for Softmax logic and float output.
    """
    N_CLASSES = params['N_CLASSES']

    code_lines = [
        '#include <cstdio>',
        '#include <cstdlib>',
        '#include <iostream>',
        '#include <cmath>', # Added cmath for expf used in Softmax (for completeness, though not strictly needed here)
        f'#include "conv_tb{num_layers}.h"',
        f'#include "conv{num_layers}.h"\n',
        'int main(void) {'
    ]

    # Buffers - Note: C_N, H_N, E_N refer to the Nth CONV layer
    code_lines.append(f' \t// Input Buffer')
    code_lines.append(f' \ttype_t *I1 = (type_t *) malloc(C1 * H1 * H1 * sizeof(type_t));')

    code_lines.append(f' \t// Weight Buffers for {num_layers} Conv Layers')
    for i in range(1, num_layers + 1):
        # W size: M_i * C_i * R_i * R_i
        code_lines.append(f' \ttype_t *W{i} = (type_t *) malloc(M{i} * C{i} * R{i} * R{i} * sizeof(type_t));')

    # --- Conv Bias Buffers ---
    code_lines.append(f' \t// Bias Buffers for {num_layers} Conv Layers')
    for i in range(1, num_layers + 1):
        # B size: M_i (Output Channels)
        code_lines.append(f' \ttype_t *B{i} = (type_t *) malloc(M{i} * sizeof(type_t));')

    # FC Weight Buffer: size of previous output (M_N * E_N * E_N) * N_CLASSES
    code_lines.append(f' \t// Weight Buffer for Final FC Layer')
    code_size_n = f'(M{num_layers} * E{num_layers} * E{num_layers})'
    code_lines.append(f' \ttype_t *W_fc = (type_t *) malloc({code_size_n} * N_CLASSES * sizeof(type_t));')

    # --- FC Bias Buffer ---
    code_lines.append(f' \t// Bias Buffer for Final FC Layer')
    code_lines.append(f' \ttype_t *B_fc = (type_t *) malloc(N_CLASSES * sizeof(type_t));')

    # Output Buffer - N_CLASSES (CHANGED to float*)
    code_lines.append(f' \t// Final Output Buffer (N_CLASSES={N_CLASSES}, stores Softmax probabilities)')
    code_lines.append(f' \tfloat *O_final = (float *) calloc(N_CLASSES, sizeof(float));')

    # Init inputs
    code_lines.append('\n \tsrand(1);')
    # Init Input Feature Map I1
    code_lines.append(f' \t// Initialize Input I1')
    code_lines.append(' \tfor(unsigned j = 0; j < C1 * H1 * H1; j++)')
    code_lines.append(' \t\tI1[j] = rand() % RANDROOF;')

    # Init Weights W_i
    code_lines.append(f'\n \t// Initialize Conv Weights')
    for i in range(1, num_layers + 1):
        code_lines.append(f' \tfor(unsigned j = 0; j < M{i} * C{i} * R{i} * R{i}; j++)')
        code_lines.append(f' \t\tW{i}[j] = rand() % RANDROOF;')

    # Init Conv Biases
    code_lines.append(f'\n \t// Initialize Conv Biases')
    for i in range(1, num_layers + 1):
        code_lines.append(f' \tfor(unsigned j = 0; j < M{i}; j++)')
        code_lines.append(f' \t\tB{i}[j] = rand() % RANDROOF;')

    # Init FC Weights
    code_lines.append(f'\n \t// Initialize FC Weights')
    code_lines.append(f' \tfor(unsigned j = 0; j < {code_size_n} * N_CLASSES; j++)')
    code_lines.append(f' \t\tW_fc[j] = rand() % RANDROOF;')

    # Init FC Biases
    code_lines.append(f'\n \t// Initialize FC Biases')
    code_lines.append(f' \tfor(unsigned j = 0; j < N_CLASSES; j++)')
    code_lines.append(f' \t\tB_fc[j] = rand() % RANDROOF;')


    # CNN Call
    code_lines.append('\n \t// Perform CNN inference (Output is Softmax probability: float*)')
    cnn_call = ' \tcnn(I1'
    for i in range(1, num_layers + 1):
        cnn_call += f', W{i}, B{i}'
    # O_final is now float*
    cnn_call += f', W_fc, B_fc, O_final);'
    code_lines.append(cnn_call)

    # --- UPDATED: Prediction Logic and Print (using float) ---
    code_lines.append('\n \t// Find the maximum probability (the prediction)')
    code_lines.append(' \tfloat max_val = O_final[0];')
    code_lines.append(' \tint predicted_class = 0;')
    code_lines.append(' \tfor (int k = 1; k < N_CLASSES; k++) {')
    code_lines.append(' \t\tif (O_final[k] > max_val) {')
    code_lines.append(' \t\t\tmax_val = O_final[k];')
    code_lines.append(' \t\t\tpredicted_class = k;')
    code_lines.append(' \t\t}')
    code_lines.append(' \t}')
    code_lines.append('\n \tprintf("--- Classification Result (Softmax Probabilities) ---\\n");')
    # UPDATED PRINTF: uses %.4f for float output
    code_lines.append(' \tprintf("Predicted Class Index: %d (with probability %.4f)\\n", predicted_class, max_val);')
    code_lines.append(' \tprintf("All Probabilities:\\n");')
    code_lines.append(' \tfor (int k = 0; k < N_CLASSES; k++) {')
    code_lines.append(' \t\tprintf(" \tClass %d: %.4f\\n", k, O_final[k]);')
    code_lines.append(' \t}')
    code_lines.append(' \tprintf("---------------------------------------------------\\n");')

    # Output to file - WRITING FLOATS
    code_lines.append(f'\n \tFILE *opf = fopen("../bin/output.bin", "wb");')
    code_lines.append(f' \tif (opf == NULL) {{')
    code_lines.append(f' \t\tperror("Error opening output.bin");')
    code_lines.append(f' \t\treturn EXIT_FAILURE;')
    code_lines.append(f' \t}}')
    # UPDATED FWRITE: writes float* output buffer
    code_lines.append(f' \tfwrite(O_final, sizeof(float), N_CLASSES, opf);')
    code_lines.append(' \tfclose(opf);')

    # Free
    code_lines.append('\n \t// Free allocated memory')
    code_lines.append(' \tif(I1) free(I1);')
    for i in range(1, num_layers + 1):
        code_lines.append(f' \tif(W{i}) free(W{i});')
        code_lines.append(f' \tif(B{i}) free(B{i});')

    code_lines.append(' \tif(W_fc) free(W_fc);')
    code_lines.append(' \tif(B_fc) free(B_fc);')
    code_lines.append(' \tif(O_final) free(O_final);')

    code_lines.append('\n \treturn EXIT_SUCCESS;\n}')
    return '\n'.join(code_lines)


def generate_conv_code(num_layers):
    """Generates the convolution and wrapper C++ implementation code, updated to include Softmax."""
    # Added math.h for expf, logf
    code = f'#include "conv{num_layers}.h"\n#include <math.h>\n\n'

    # Add ReLU function
    code += "type_t relu(type_t x) {\n"
    code += " \treturn (x > 0) ? x : 0;\n"
    code += "}\n\n"

    # --- ADDED: Softmax function (uses float for output and internal calculation) ---
    code += "void softmax(type_t input[N_CLASSES], float output[N_CLASSES]) {\n"
    code += " \tfloat sum = 0.0f;\n"
    code += " \tfloat max_val = (float)input[0];\n"
    code += " \t// Find max for numerical stability\n"
    code += " \tfor (int k = 1; k < N_CLASSES; k++) {\n"
    code += " \t\tif ((float)input[k] > max_val) max_val = (float)input[k];\n"
    code += " \t}\n"
    code += " \t// Calculate exponentials and sum (subtract max for stability)\n"
    code += " \tfor (int k = 0; k < N_CLASSES; k++) {\n"
    code += " \t\toutput[k] = expf((float)input[k] - max_val);\n"
    code += " \t\tsum += output[k];\n"
    code += " \t}\n"
    code += " \t// Normalize to get probabilities\n"
    code += " \tfor (int k = 0; k < N_CLASSES; k++) {\n"
    code += " \t\toutput[k] /= sum;\n"
    code += " \t}\n"
    code += "}\n\n"
    # ------------------------------------------------------------------------

    # Generate N standard convolution layer functions (conv1 to convN)
    for i in range(1, num_layers + 1):
        # Conv function body
        code += f"void conv{i}(type_t I{i}[C{i} * H{i} * H{i}], type_t W{i}[M{i} * C{i} * R{i} * R{i}], type_t B{i}[M{i}], type_t O{i}[M{i} * E{i} * E{i}]) {{\n"
        code += f" \tfor(int m = 0; m < M{i}; m++) {{\n"
        code += f" \t\tfor(int y = 0; y < E{i}; y++) {{\n"
        code += f" \t\t\tfor(int x = 0; x < E{i}; x++) {{\n"

        # Initialize output with bias term B{i}[m]
        code += f" \t\t\t\tO{i}[x + (y + (m * E{i})) * E{i}] = B{i}[m];\n"

        # HLS Pipelining for the inner MAC operation
        code += f" \t\t\t\t#pragma HLS PIPELINE II=1\n"

        code += f" \t\t\t\tfor(int c = 0; c < C{i}; c++) {{\n" # Channel loop
        code += f" \t\t\t\t\tfor(int k = 0; k < R{i}; k++) {{\n" # Kernel height loop
        code += f" \t\t\t\t\t\tfor(int l = 0; l < R{i}; l++) {{\n" # Kernel width loop

        code += f" \t\t\t\t\t\t\t// Zero-padding implementation\n"
        code += f" \t\t\t\t\t\t\tint h1 = y * S{i} - PAD{i} + k;\n"
        code += f" \t\t\t\t\t\t\tint h2 = x * S{i} - PAD{i} + l;\n"
        code += f" \t\t\t\t\t\t\ttype_t val = (h1 < 0 || h1 >= H{i} || h2 < 0 || h2 >= H{i}) ? 0 : I{i}[h2 + (h1 + (c * H{i})) * H{i}];\n"
        code += f" \t\t\t\t\t\t\tO{i}[x + (y + (m * E{i})) * E{i}] += val * W{i}[l + (k + (c + (m * C{i})) * R{i}) * R{i}];\n"

        code += f" \t\t\t\t\t\t}}\n" # Close l
        code += f" \t\t\t\t\t}}\n" # Close k
        code += f" \t\t\t\t}}\n" # Close c
        code += f" \t\t\t}}\n" # Close x
        code += f" \t\t}}\n" # Close y
        code += f" \t}}\n" # Close m
        code += f"}}\n\n"

    # Final Dedicated FC layer for classification
    input_size_n = f"M{num_layers} * E{num_layers} * E{num_layers}"

    # UPDATED: FC layer no longer applies ReLU. It outputs raw scores (type_t).
    code += f"void fc_layer(type_t input[{input_size_n}], type_t W_fc[{input_size_n} * N_CLASSES], type_t B_fc[N_CLASSES], type_t output[N_CLASSES]) {{\n"
    code += f" \t// Flatten the input and perform N_CLASSES dot products\n"
    code += f" \tfor (int k = 0; k < N_CLASSES; k++) {{\n"

    # Initialize output with bias term B_fc[k]
    code += f" \t\toutput[k] = B_fc[k];\n"

    code += f" \t\tfor (int i = 0; i < {input_size_n}; i++) {{\n"
    code += f" \t\t\t// W_fc is indexed by [output_class][input_feature]\n"
    code += f" \t\t\toutput[k] += input[i] * W_fc[i + k * {input_size_n}];\n"
    code += f" \t\t}}\n"
    # REMOVED: ReLU is NOT applied before Softmax
    code += f" \t}}\n"
    code += f"}}\n\n"

    # Wrapper CNN function
    code += "void cnn(type_t *input"
    for i in range(1, num_layers + 1):
        code += f", type_t *W{i}, type_t *B{i}"
    # Final output type is float* for probabilities
    code += ", type_t *W_fc, type_t *B_fc, float *output) {\n"
    code += "#pragma HLS DATAFLOW\n"

    # Intermediate buffers for dataflow
    for i in range(1, num_layers + 1):
        code += f" \tstatic type_t O{i}[M{i} * E{i} * E{i}];\n"

    # Internal buffer for raw FC scores (unsigned type_t)
    code += f" \tstatic type_t O_raw[N_CLASSES];\n\n"

    # Chaining the N convolution layers (remains the same)
    for i in range(1, num_layers + 1):
        if i == 1:
            code += f" \tconv1(input, W1, B1, O1);\n"
            code += f" \t// Apply ReLU to intermediate output O1\n"
            code += f" \tfor (int j = 0; j < M1 * E1 * E1; j++) O1[j] = relu(O1[j]);\n"
        else:
            code += f" \tconv{i}(O{i-1}, W{i}, B{i}, O{i});\n"
            code += f" \t// Apply ReLU to intermediate output O{i}\n"
            code += f" \tfor (int j = 0; j < M{i} * E{i} * E{i}; j++) O{i}[j] = relu(O{i}[j]);\n"

    # Final FC layer
    code += "\n \t// Final Layer: Fully Connected (FC) Classification Head - outputs raw scores (O_raw)\n"
    # FC outputs raw scores (type_t) into O_raw
    code += f" \tfc_layer(O{num_layers}, W_fc, B_fc, O_raw);\n"

    # Softmax layer
    code += "\n \t// Softmax Layer: Convert raw scores (O_raw) to probability distribution (output)\n"
    # Softmax takes O_raw (type_t) and outputs probabilities (float) into 'output'
    code += f" \tsoftmax(O_raw, output);\n"

    code += "}\n"
    return code


def generate_convh_code(num_layers, params):
    """Generates the convolution header C++ code with calculated parameters, updated for Softmax output."""
    code = "#ifndef CONV_H\n#define CONV_H\n\n#include <cstddef>\ntypedef unsigned type_t;\n\n"

    # Global Parameters
    code += f"// ** CLASSIFICATION PARAMETERS **\n"
    code += f"const size_t N_CLASSES = {params['N_CLASSES']};\n"

    # **ADDED**: Bias Constant
    code += f"const size_t BIAS_ON = 1; // 1: Bias is included\n\n"

    # Input Parameters (Defined only once)
    code += "// ** INPUT PARAMETERS **\n"
    code += f"const size_t C1 = {params['C1']}; // Input channels\n"
    code += f"const size_t H1 = {params['H1']}; // Input size H x H\n\n"

    for i in range(1, num_layers + 1):
        code += f"// ** CONV LAYER {i} **\n"

        if i > 1:
            code += f"const size_t C{i} = {params[f'C{i}']};\n" # Input Channels of current layer
            code += f"const size_t H{i} = {params[f'H{i}']};\n" # Input Size of current layer
        else:
            pass

        code += f"const size_t M{i} = {params[f'M{i}']};\n" # Output Channels
        code += f"const size_t R{i} = {params[f'R{i}']};\n" # Kernel Size
        code += f"const size_t S{i} = {params[f'S{i}']};\n" # Stride
        code += f"const size_t E{i} = {params[f'E{i}']};\n" # Output Size

        # Calculated parameters
        code += f"const size_t F{i} = ((E{i} * S{i} + R{i} - 1) < H{i}) ? H{i} : (E{i} * S{i} + R{i} - 1);\n"
        code += f"const size_t PAD{i} = (F{i} - H{i}) / 2;\n\n"

    # Function prototypes
    for i in range(1, num_layers + 1):
        # UPDATED PROTOTYPE: Added B{i}
        code += f"void conv{i}(type_t I{i}[C{i} * H{i} * H{i}], type_t W{i}[M{i} * C{i} * R{i} * R{i}], type_t B{i}[M{i}], type_t O{i}[M{i} * E{i} * E{i}]);\n"

    input_size_n = f"M{num_layers} * E{num_layers} * E{num_layers}"
    # UPDATED PROTOTYPE: FC output is type_t raw scores
    code += f"void fc_layer(type_t input[{input_size_n}], type_t W_fc[{input_size_n} * N_CLASSES], type_t B_fc[N_CLASSES], type_t output[N_CLASSES]);\n"

    # --- ADDED: Softmax prototype ---
    code += f"void softmax(type_t input[N_CLASSES], float output[N_CLASSES]);\n"

    code += "\nvoid cnn(type_t *input"
    for i in range(1, num_layers + 1):
        # UPDATED PROTOTYPE: Added W{i} and B{i}
        code += f", type_t *W{i}, type_t *B{i}"
    # UPDATED FINAL ARGUMENT TYPE: float* for Softmax probabilities
    code += ", type_t *W_fc, type_t *B_fc, float *output);\n"

    code += "\n#endif"
    return code


# --- 2. Parameter Calculation and GUI Setup (Unchanged) ---

def calculate_output_size(H_in, R, S, P):
    """Calculates output size E based on HLS-style padding/stride. We enforce E = ceil(H_in / S)"""
    return math.ceil(H_in / S)

def generate_parameter_widgets(num_layers):
    """Generates the main parameter input widgets."""

    # Global Classification Parameter
    n_classes_widget = widgets.IntText(value=10, description='N_CLASSES:', min=1, style={'description_width': 'initial'})

    # **ADDED** (Disabled as bias is mandatory now)
    bias_on_widget = widgets.IntText(value=1, description='Bias ON (1=Yes):', min=1, max=1, style={'description_width': 'initial'}, disabled=True)

    # Input Feature Map (Layer 1 Input)
    c1_widget = widgets.IntText(value=3, description='Input C1:', min=1, style={'description_width': 'initial'})
    h1_widget = widgets.IntText(value=32, description='Input H1:', min=1, style={'description_width': 'initial'})

    widgets_list = [
        widgets.HTML(value="<h3>Classification Parameters:</h3>"),
        n_classes_widget,
        bias_on_widget, # Display the new widget
        widgets.HTML(value="<h3>Input Feature Map (Layer 1 Input):</h3>"),
        c1_widget,
        h1_widget,
        widgets.HTML(value=f"<h3>Convolution Layers (1 to {num_layers}):</h3>")
    ]

    layer_widgets = {}

    for i in range(1, num_layers + 1):
        # Channels Out (M_i) - user defines
        m_i = widgets.IntText(value=32, description=f'L{i} M (Ch Out):', min=1, style={'description_width': 'initial'})

        # Kernel Size (R_i) - user defines
        r_i = widgets.IntText(value=3, description=f'L{i} R (Kernel):', min=1, style={'description_width': 'initial'})

        # Stride (S_i) - user defines
        s_i = widgets.IntText(value=1, description=f'L{i} S (Stride):', min=1, style={'description_width': 'initial'})

        layer_widgets[i] = (m_i, r_i, s_i)

        widgets_list.append(widgets.VBox([
            widgets.HTML(value=f"<h4>Conv Layer {i}</h4>"),
            widgets.HBox([m_i, r_i, s_i])
        ]))

    # Wrap parameter widgets in a VBox container
    params_vbox = widgets.VBox(widgets_list)

    # UPDATED RETURN
    return params_vbox, layer_widgets, c1_widget, h1_widget, n_classes_widget

def collect_and_calculate_params(num_layers, layer_widgets, c1_widget, h1_widget, n_classes_widget):
    """Collects user input and calculates dependent parameters (C_i, H_i, E_i). (No change needed)"""

    params = {}

    # Global Parameter
    params['N_CLASSES'] = n_classes_widget.value

    # Initial Input
    params['C1'] = c1_widget.value
    params['H1'] = h1_widget.value

    H_prev = params['H1']
    M_prev = params['C1']

    for i in range(1, num_layers + 1):
        m_i, r_i, s_i = layer_widgets[i]

        # C_i (Input Channels) = M_{i-1} (Output Channels of prev layer)
        # H_i (Input Size) = E_{i-1} (Output Size of prev layer)
        params[f'C{i}'] = M_prev
        params[f'H{i}'] = H_prev

        # User Defined Parameters
        params[f'M{i}'] = m_i.value
        params[f'R{i}'] = r_i.value
        params[f'S{i}'] = s_i.value

        # Calculated Output Size (E_i) - Enforcing 'Same' or 'Half' Padding
        params[f'E{i}'] = calculate_output_size(params[f'H{i}'], params[f'R{i}'], params[f'S{i}'], 0) # P is calculated later

        # For next layer's input
        H_prev = params[f'E{i}']
        M_prev = params[f'M{i}']

        # Validation
        if params[f'R{i}'] > params[f'H{i}'] and params[f'H{i}'] > 1:
            raise ValueError(f"Layer {i}: Kernel size R{i}={params[f'R{i}']} must be less than or equal to input size H{i}={params[f'H{i}']}.")
        if params['N_CLASSES'] < 1:
            raise ValueError("N_CLASSES must be 1 or greater.")

    return params


# --- 3. GUI and Execution (Unchanged) ---

layer_slider = widgets.IntSlider(value=3, min=1, max=5, step=1, description='CNN Layers:', continuous_update=False, style={'description_width': 'initial'})
generate_button = widgets.Button(description='Generate Code (with Softmax)', button_style='success')
output_area = widgets.Output()
code_controls_vbox = widgets.VBox() # Container for the dynamically changing parameter widgets

# Initial setup of parameter widgets
param_vbox, layer_widgets_map, c1_input, h1_input, n_classes_input = generate_parameter_widgets(layer_slider.value)
code_controls_vbox.children = (param_vbox,) # Place generated widgets into the container

def update_widgets(change):
    """Update parameter widgets when the number of layers changes."""
    global layer_widgets_map, c1_input, h1_input, n_classes_input

    num_layers = layer_slider.value
    # Generate a new set of widgets
    # Note: the return value of generate_parameter_widgets matches the required global updates.
    new_param_vbox, layer_widgets_map, c1_input, h1_input, n_classes_input = generate_parameter_widgets(num_layers)

    # Replace the content of the VBox container
    code_controls_vbox.children = (new_param_vbox,)

    with output_area:
        clear_output()
        display(Markdown(f"Parameters updated for **{num_layers} layers**. Click 'Generate Code'."))

layer_slider.observe(update_widgets, names='value')

def on_button_click(b):
    """Code generation main logic. Updates paths to use SRC/BIN structure."""
    with output_area:
        clear_output()

        num_layers = layer_slider.value
        i = num_layers

        try:
            # 1. Collect and Calculate Parameters
            params = collect_and_calculate_params(num_layers, layer_widgets_map, c1_input, h1_input, n_classes_input)

            # 2. Setup Folder (Dynamically named based on number of layers)
            # Source folder is now nested: ./repo/generatedCNN_NLayers/src
            root_folder = f'./repo/generatedCNN_{i}Layers'
            source_folder = os.path.join(root_folder, 'src')
            bin_folder = os.path.join(root_folder, 'bin') # Ensure bin folder is created
            os.makedirs(source_folder, exist_ok=True)
            os.makedirs(bin_folder, exist_ok=True) # Ensure bin folder is also created by Python for clean setup

            # 3. Generate Files

            # All files are written into the 'src' subfolder
            with open(f'{source_folder}/Makefile{i}', 'w') as f: f.write(generate_makefile_code(i))
            with open(f'{source_folder}/conv_tb{i}.cpp', 'w') as f: f.write(generate_testbench_code(i, params))
            with open(f'{source_folder}/conv_tb{i}.h', 'w') as f: f.write(generate_testbench_header_code(i))
            with open(f'{source_folder}/conv{i}.h', 'w') as f: f.write(generate_convh_code(i, params))
            with open(f'{source_folder}/conv{i}.cpp', 'w') as f: f.write(generate_conv_code(i))

            display(Markdown(f'## ✅ Success! All files for **{i} CONV layers** + **1 FC layer (with Softmax)** generated in `{source_folder}`'))

            # Print a summary of the calculated parameters
            param_summary = ["| Layer | Type | C (In) | H (In) | M (Out) | R (K) | S (Str) | E (Out) | Activation |"]
            param_summary.append("|---|---|---|---|---|---|---|---|---|")

            # Initial Input
            param_summary.append(f"| Input | Image | {params['C1']} | {params['H1']} | N/A | N/A | N/A | N/A | No |")

            for j in range(1, num_layers + 1):
                param_summary.append(f"| {j} | CONV | {params[f'C{j}']} | {params[f'H{j}']} | {params[f'M{j}']} | {params[f'R{j}']} | {params[f'S{j}']} | {params[f'E{j}']} | **ReLU** |")

            # Final FC layer summary
            input_size = params[f'M{num_layers}'] * params[f'E{num_layers}'] * params[f'E{num_layers}']
            param_summary.append(f"| N+1 | **FC** | {input_size} (flat) | N/A | **{params['N_CLASSES']}** | N/A | N/A | N/A | **Softmax** |")

            display(Markdown('### Calculated CNN Parameters\n' + '\n'.join(param_summary)))

            display(Markdown('### Execution Instructions (SRC/BIN Structure)'))
            display(Markdown(
                f"1. **Navigate to the Source:** `cd {source_folder}`\n"
                f"2. **Compile:** `make -f Makefile{i} conv{i}` (Note: The Makefile now uses `-lm` for the math library!)\n"
                f"3. **Run:** Go to the parent directory: `cd ../` and execute the binary: `./bin/conv{i}`\n"
                f"The executable (`conv{i}`), the classification probabilities (printed to console), and the raw output file (`output.bin`, containing floats) are in the **`{root_folder}/bin/`** folder."
            ))

        except ValueError as e:
            display(Markdown(f'## ❌ Error: {e}'))

        except Exception as e:
            display(Markdown(f'## ❌ An unexpected error occurred: {e}'))


generate_button.on_click(on_button_click)

# FINAL DISPLAY: Display the components only once
display(layer_slider, code_controls_vbox, generate_button, output_area)